In [16]:
import numpy as np
import pandas as pd
from numpy import pi, r_
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import rc
from scipy import optimize
from scipy.optimize import minimize
import re
import sys
import subprocess
import math

In [17]:
font = {'size': 16}
rc('font', **font)

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams["lines.linewidth"]=2

plotting=True # plots should be created ?
subplots=True
pico=1.*10**(-12) # picoseconds

In [18]:
# file with QUADRIC analysis of the protein backbone in order to define
# global tumbling (general or specific for each methyl) has to be done before running this script

inputpath = "."
variant='CI2_WT' 
tumbling = "iso" # iso/ani

outputpath = inputpath+"/"+variant+"/"+tumbling

## Creat output folder
COMMAND = f"mkdir -p {variant}/{tumbling}"
subprocess.call(COMMAND, shell=True)

# Read in input files
tumbling_file=inputpath+"/analysis_"+tumbling+"_"+variant
inputfile_times=inputpath+"/"+variant+"_sch_times.csv"
relax_times_exp = pd.read_csv(inputfile_times, sep=",", index_col=[0])


methID_dict = {"Ala":"ALA", "Ile":"ILE", "Leu":"LEU", "Met":"MET", "Thr":"THR", "Val":"VAL",
               "Cb":"-CB", "Hb":"HB", "Cg1":"-CG1","Cg2":"-CG2", "Hg1":"HG1","Hg2":"HG2","Cd1":"-CD1", "Hd1":"HD1",
               "Cd2":"-CD2", "Hd2":"HD2", "Ce":"-CE", "He":"HE"}



In [86]:
nfields=2 # 600, 750 MHz
nrates=4 # R1, R2, R3, R4
nmodels=2 # LS2, LS3
njws=3 # Jomega0, Jomega_D, Jomega_2D 

# Larmor frquency of deuterium
omega_D_field1=2.*pi*92.1298227037992*1000000 # Hr @600 MHz
omega_D_field2=2.*pi*115.138869111022*1000000 # Hz @750 MHz
# frequencies of spectral density used for the fit / usually frequencies measured in NMR

# create array with five frequences  
measuredfreq_field1=np.array([i*omega_D_field1 for i in [0,1,2]])
measuredfreq_field2=np.array([i*omega_D_field2 for i in [0,1,2]])

measuredfreq = np.array([measuredfreq_field1[0], measuredfreq_field2[0],measuredfreq_field1[1], measuredfreq_field2[1],
               measuredfreq_field1[2], measuredfreq_field2[2]])

frequency = np.linspace(0, 2*10**9, 100)


In [93]:
class methyl_MF:
    
        pre=2*np.pi*167000 # prefactor based on physical constants
        A = 3./16.*pre**2 # prefactor A
        B = 1./32.*pre**2 # prefactor B
    
    
        def __init__(self,methyl_numb=0,rates=[],rates_err=[]):
            
            self.methyl_numb = methyl_numb
            self.rates = []
            self.rares_err = []
        
        def get_rates (self,methyl_numb,relax_times_exp):
    
            times=relax_times_exp.iloc[methyl_numb][::2].values # in ms
            self.rates = 1/(times*0.001) 
            
            return self.rates
            
        def get_rates_err (self,methyl_numb,relax_times_exp):
    
            errors = relax_times_exp.iloc[methyl_numb][1::2].values # in ms
            self.rares_err = np.divide(errors/0.001, times**2) 
            
            return self.rares_err
        
        def get_rates_and_err (self,methyl_numb,relax_times_exp):
    
            times=relax_times_exp.iloc[methyl_numb][::2].values # in ms
            errors = relax_times_exp.iloc[methyl_numb][1::2].values # in ms
            self.rates = 1/(times*0.001) 
            self.rares_err = np.divide(errors/0.001, times**2) 
            
            return self.rates, self.rares_err

        
        # methods to calculate spectral density points from methyl relaxation rates

        def J_omega_D(self,R1,R2,R3):
            return R3/(3*methyl_MF.A) - R1*0 - R2*0
        
        def J_2omega_D(self,R1,R2,R3):
            return (R1/methyl_MF.A - R3/(3*methyl_MF.A))/4 - R2*0
        def J_2omega_D_b(self,R2,R3,R4):
            return (R4/methyl_MF.A - R3/(3*methyl_MF.A))/2 - R2*0  
        
        def J_0(self,R1,R2,R3):
            return ( R2/methyl_MF.B - 5*R3/methyl_MF.A - 1.5*R1/methyl_MF.A + 0.5*R3/methyl_MF.A ) /9
        def J_0_b(self,R2,R3,R4):
            return ( R2/methyl_MF.B - 5*R3/methyl_MF.A - 3*R4/methyl_MF.A + R3/methyl_MF.A ) /9
        
        
        # methods to calculate methyl relaxation rates from spectral density points
        def R1(self,x,y,z):
            return 3./16.*methyl_MF.pre**2*(0*x+ 1*y+4*z)
        def R2(self,x,y,z):
            return 1./32.*methyl_MF.pre**2*(9*x+15*y+6*z)
        def R3(self,x,y,z):                            
            return 3./16.*methyl_MF.pre**2*(0*x+ 3*y+0*z)
        def R4(self,x,y,z):
            return 3./16.*methyl_MF.pre**2*(0*x+ 1*y+2*z)
        

In [94]:
m = methyl_MF()

In [95]:
rates = m.get_rates(0,relax_times_exp)

In [96]:
rates

array([27.07925339, 74.43677415, 24.18129978, 19.14059494, 24.291437  ,
       74.65951528, 20.26205319, 15.30364498])

In [97]:
m.A

206440047856.4858

In [98]:
#Jzero_field1=J_0(Rates[0],Rates[1],Rates[2]) 
#Jzero_field2=J_0(Rates[4],Rates[5],Rates[6]) # J_0(R1,R2,R3)

Jzero_field1 = m.J_0(rates[0],rates[1],rates[2])
Jw_D_field1=m.J_omega_D(rates[0],rates[1],rates[2])
J2w_D_field1=m.J_2omega_D(rates[0],rates[1],rates[2])

In [99]:
print (Jzero_field1,Jw_D_field1,J2w_D_field1)

1.5995276911477115e-10 3.9044910826487544e-11 2.3031892709764054e-11


In [100]:
rate1_field1=m.R1(Jzero_field1,Jw_D_field1,J2w_D_field1)

In [101]:
rate1_field1

27.079253392488862